# Coll Docu Scraper

In [1]:
# # Libraries required
# !pip install selenium
# !pip install webdriver-manager
# !pip install pillow
# !pip install img2pdf
# !pip install xlsxwriter
# !pip install html5lib
# !pip install bs4

In [1]:
import os
import pandas as pd
import json
import pickle
import urllib.request
import glob
from PIL import Image, ImageSequence
import img2pdf
import numpy as np
import shutil
import html5lib

In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup
import requests

PATH = 'C:\Program Files (x86)\chromedriver.exe'
DOWNLOAD_PATH = 'C:/Users/ebryaga/Downloads'

In [3]:
# To ignore warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
driver = webdriver.Chrome(PATH)

driver.get('http://edmdote.bakerhughes.com:8080/ged-html/input.thtml?tipol=ALL')

# search = driver.find_element_by_name("matr_macchina")
# search.send_keys("G06500")

# print(driver.title)
# driver.quit()

## Sign in in Pop up window to Coll DOcu using SSO

In [18]:
driver.get('http://edmdote.bakerhughes.com:8080/ged-html/input.thtml?tipol=ALL')

In [6]:
table_names = driver.find_element('xpath', '/html/body/form/p[1]/table/tbody/tr[3]/td[2]/select').text.split('\n')
table_names_to_full_name = {x.split('-')[1]:x for x in table_names}
table_names_to_num = {x.split('-')[1]:x.split('-')[0] for x in table_names}

In [22]:
def search_documents(x):
    '''
    This function enables you to serach documents by Serial Numbers.
    
    Args:
        x - serial number
        
    Returns:
        Widnow is switched for searched SN documentation.
    '''
    
    search_window = driver.window_handles[0]   #  Defining window/page
    driver.switch_to.window(search_window)     # Switching between widnows/pages
#     search = driver.find_element_by_name("matr_macchina")    # Old version
    search = driver.find_element("name", "matr_macchina")
    search.clear()     # Clear selected textbox
    search.send_keys(x) # Fill selected textbox with x - function input
    
    database = driver.find_element('xpath', '/html/body/form/p[1]/table/tbody/tr[1]/td[2]/select')
    database.send_keys("ALL")
    
    search_limit = driver.find_element('xpath', '/html/body/form/p[3]/input')
    search_limit.clear()
    search_limit.send_keys("10000")
    
#     submit = driver.find_element('xpath', '/html/body/form/center[1]/p/a[1]/img')
    submit = driver.find_element('xpath', '/html/body/form/center[2]/a[1]/button')
    submit.click()
    
    #switch to results
    new_window = driver.window_handles[1]
    driver.switch_to.window(new_window)

In [32]:
def get_tables(x):
    '''
    This function creates Excel spreadsheet with overview of found documents in Coll Docu for given SN and also
    saving those tables in json format.
    This function is not executing searching process again. The input SNs is needed only for file naming purposes.
    
    Args:
        x - serial number
        
    Returns:
        {SN}.json - saving dict with tables
        SN_{SN_num}.xlsx  - spreadsheet with tables (same as json, but saved in different format)
    '''
    search_window = driver.window_handles[0]
    new_window = driver.window_handles[1]
    driver.switch_to.window(new_window)
    source = driver.page_source
    
    # Get tables for result page
    try:
        df = pd.read_html(source.replace('100%','1'))
        my_dict = {}
        for d in df:
            if type(d.iloc[0,0]) != np.float64:
                my_key = d.iloc[0,0].replace('\xa0',' ').split(' ')[1]
                if my_key in my_dict.keys():
                    df_1 = my_dict[my_key]
                    my_value = d.iloc[1:,:]
                    my_value.columns = my_value.iloc[0]
                    my_value = my_value.iloc[1:,:]
                    df_full = df_1.append(my_value)
                    my_dict[my_key] = df_full
                else:
                    my_value = d.iloc[1:,:]
                    my_value.columns = my_value.iloc[0]
                    my_value = my_value.iloc[1:,:]
                    my_dict[my_key] = my_value
                
        # Saving to json
        with open(f'{x}.json', 'wb') as fp:
            pickle.dump(my_dict, fp)

        # Saving to excel   
        writer = pd.ExcelWriter(f'SN_{x}.xlsx', engine='xlsxwriter')
        for key, value in my_dict.items():
            value.to_excel(writer, sheet_name=key)
        writer.save()
    
    except ValueError:
        print('Whoops')
        
    driver.switch_to.window(search_window)

In [34]:
def get_att_links(x):
    '''
    This function creates a list of links (xpaths) for attachments.
    
    Args:
        x - serial number (only for namin purposes?)
        
    Returns:
        links - list, containing links (xpaths) for ALL attachments
    
    '''
    search_window = driver.window_handles[0]
    new_window = driver.window_handles[1]
    driver.switch_to.window(new_window)
    source = driver.page_source
    try:
        df = pd.read_html(source.replace('100%','1'))
        my_attachments = {}        #{'Table 17': [indexy, bez duplikatow]}
        table = 1
        for d in df:
            if type(d.iloc[0,0]) != np.float64:
                my_key = d.iloc[0,0].replace('\xa0',' ').split(' ')[1]
                my_value = d.iloc[1:,:]
                my_value.columns = my_value.iloc[0]
                my_value = my_value.iloc[1:,:]
            
            
            
            if int(my_key) == 17:
                a = my_value[['N.FRAMES', 'JOB.', 'JOB.REV.']]
                att_indexes = [2] + a[a.shift(-1) != a].dropna(how='all').index.to_list()
                my_attachments[table] = att_indexes
    #             my_attachments[my_key] = my_value.drop_duplicates(subset=['N.FRAMES', 'JOB.', 'JOB.REV.']).index.to_list()
            else:
                a = my_value[['N.FRAMES', 'JOB.']]
                att_indexes = [2] + a[a.shift(-1) != a].dropna(how='all').index.to_list()
                att_indexes = list(set(att_indexes))
                my_attachments[table] = att_indexes

            table += 1
    
        links = []
        for key, value in my_attachments.items():
            for y in value:
                updated_value = int(y) + 1
                link = f'/html/body/form/b[2]/table[{key}]/tbody/tr[{updated_value}]/td[1]/a'
                links.append(link)
                
        return links
    
    except ValueError:
        print('Whoops')
        return None

In [35]:
# Full_version

def download_filter_attachements(x, links):
    '''
    This function downloads files based on list of links (xpaths) and moves it to dedicated folder for each SN.
    
    Args:
        x - serial number (only for namin purposes?)
        links - list of xpaths, this is a result from "get_att_links" function
        
    Returns:
        download file - depneds of type of donloaded file:
                        pdf - normal download
                        tiff - download and covert to pdf format
    
    '''   
    i = 1
    tables = []
    jobs = []
    pages = []
    org_directory = os.getcwd()
#     attachments = driver.find_elements_by_xpath('/html/body/form/b[2]//a')[:-1]
    for att_path in links:
        print(att_path)
        attachment = driver.find_element('xpath', att_path)
        attachment.click()
        att_window = driver.window_handles[-1]
        driver.switch_to.window(att_window)
        try:
            download = driver.find_element('xpath', '/html/body/form/center[2]/a')
        except:
            download = driver.find_element('xpath', '/html/body/form/a')
        table = driver.find_element('xpath', '/html/body/form/table/tbody/tr[1]/td[2]').text
        job = driver.find_element('xpath', '/html/body/form/table/tbody/tr[5]/td[2]').text
        page = driver.find_elements('xpath', '/html/body/form/table/tbody/tr/td[2]')[-1].text
            

        table = table_names_to_num[table]
        if int(table) == 17:
            reapir_job = driver.find_element('xpath', '/html/body/form/table/tbody/tr[6]/td[2]').text
       
        if len(tables) == 0:
            path = f'{org_directory}\\{table}'
            os.mkdir(path)
            os.chdir(path)
        if (len(tables) > 0) and table != tables[-1]:
            path = f'{org_directory}\\{table}'
            if not os.path.exists(path):
                os.mkdir(path)
                i = 1
            os.chdir(path)       
        if page == 'COLL/SERM':
            print("Page not found")
            page = "0"
        
        
        
        if (len(tables) == 0) or (len(tables) >= 1 and (table, job, page) != (tables[-1], jobs[-1], pages[-1])):
            tables.append(table)
            jobs.append(job)
            pages.append(page)
            try:
                download = driver.find_element('xpath','/html/body/form/center[2]/a')
            except:
                download = driver.find_element('xpath','/html/body/form/a')
            current_url = driver.current_url
            download.click()
            
            if int(table) == 17:
#                 reapir_job = driver.find_element_by_xpath('/html/body/form/table/tbody/tr[6]/td[2]').text
                if driver.current_url != current_url:
                    try:
                        download_file(driver.current_url, f'SN_{x}_{table}_{job}_RJ{reapir_job}_{page}p_{i}')
                    except:
                        print('Download went wrong', x, table, job)
#                     download_file(driver.current_url, f'SN_{x}_{table}_{job}_RJ{reapir_job}_{page}p_{i}')
                    i += 1   
                else:
                    downloaded = False
                    while not downloaded:
                        try:
                            list_of_files = glob.glob(f'{DOWNLOAD_PATH}/*')
                            latest_file = max(list_of_files, key=os.path.getctime)
                            norm_latest_file = str(os.path.normpath(latest_file))
                            downloaded = True
                        except FileNotFoundError:
                            time.sleep(2)
                                                                    

                    while (norm_latest_file.split('.')[-1] != 'tiff'):
                        time.sleep(2)
                        list_of_files = glob.glob(f'{DOWNLOAD_PATH}/*') # * means all if need specific format then *.csv
                        try:
                            latest_file = max(list_of_files, key=os.path.getctime)
                            norm_latest_file = str(os.path.normpath(latest_file))
                            print(norm_latest_file)
                            if norm_latest_file.split('.')[-1] == 'zip':
                                    break
                        except FileNotFoundError:
                            time.sleep(1)
                    if (norm_latest_file.split('.')[-1] == 'tiff'):
                        with open(f'{os.getcwd()}\\SN_{x}_{table}_{job}_RJ{reapir_job}_{page}p_{i}.pdf',"wb") as f:
                            f.write(img2pdf.convert(norm_latest_file))
                    
                    # jesli plik jest innego rozszerzenia niz tiff, to po prostu go przenosi to odpowiedniego folderu
                    else:
                        src = norm_latest_file
                        file = norm_latest_file.split('\\')[-1]
                        dst = f'{os.getcwd()}\\{file}'
                        shutil.move(src,dst)
                        
                    i += 1  
            elif int(table) in [14, 15]:                    # było "else:" ale chcemy, zeby bral tylko 14, 15 i 17
                if driver.current_url != current_url:
                    try:
                        download_file(driver.current_url, f'SN_{x}_{table}_{job}_{page}p_{i}')
                    except:
                        print('Download went wrong', x, table, job)
                    i += 1   
                else:
                    
                    downloaded = False
                    while not downloaded:
                        try:
                            list_of_files = glob.glob(f'{DOWNLOAD_PATH}/*')
                            latest_file = max(list_of_files, key=os.path.getctime)
                            norm_latest_file = str(os.path.normpath(latest_file))
                            downloaded = True
                        except FileNotFoundError:
                            time.sleep(2)
                    while (norm_latest_file.split('.')[-1] != 'tiff'):
                        time.sleep(2)
                        list_of_files = glob.glob(f'{DOWNLOAD_PATH}/*') # * means all if need specific format then *.csv
                        try:
                            latest_file = max(list_of_files, key=os.path.getctime)
                            norm_latest_file = str(os.path.normpath(latest_file))
                            print(norm_latest_file)
                            if norm_latest_file.split('.')[-1] == 'zip':
                                    break
                        except FileNotFoundError:
                            time.sleep(1)
                    if (norm_latest_file.split('.')[-1] == 'tiff'):
                        with open(f'{os.getcwd()}\\SN_{x}_{table}_{job}_{page}p_{i}.pdf',"wb") as f:
                            f.write(img2pdf.convert(norm_latest_file))
                    
                    # jesli plik jest innego rozszerzenia niz tiff, to po prostu go przenosi to odpowiedniego folderu
                    else:
                        src = norm_latest_file
                        file = norm_latest_file.split('\\')[-1]
                        dst = f'{os.getcwd()}\\{file}'
                        shutil.move(src,dst)
                        
                    i += 1
                        
        driver.switch_to.window(driver.window_handles[1])    

In [36]:
# Old function, nto in used now

def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()

## Preparing input - list od customers and SNs
- Example input file can be found in  01_06.csv file

In [12]:
df = pd.read_csv('01_06.csv')

In [13]:
df_2 = df[['E_Equipment Serial Number', 'E_OEM Nameplate SN', 'P_Site Customer Name', 'E_Equipment Code', 'E_Equipment Job Number']].copy()

In [14]:
df_2.columns = ['SN_EQ', 'SN_OEM', 'Customer', 'Frame', 'Job']

In [15]:
df_2 = df_2.sort_values(by='Customer').reset_index()

In [16]:
customer_list = df_2['Customer'].unique()

In [ ]:
# Full download

for customer in customer_list[:5]:
    customer_folder = customer.replace('/','').replace('\\','')
    new_folder = f'C:\\Users\\ebryaga\\Desktop\\Coll_Docu_Test\\{customer_folder}'
    try:
        os.mkdir(new_folder)
    except:
        print('Folder exists')
    os.chdir(new_folder)
    df_temp = df_2[df_2['Customer'] == customer]
    sn_list = df_temp['SN_OEM'].to_list()
    for x in sn_list:
        new_folder = f'C:\\Users\\ebryaga\\Desktop\\Coll_Docu_Test\\{customer_folder}\\SN_{x}'
        try:
            os.mkdir(new_folder)
        except:
            print('Folder exists')
        os.chdir(new_folder)
        search_documents(x)
        get_tables(x)
        att_links = get_att_links(x)
        if att_links != None:
            try:
                download_filter_attachements(x, att_links)
            except:
                print(f'Cannot Download {att_links}')

        # Back to main folder
        os.chdir(f'C:\\Users\\ebryaga\\Desktop\\Coll_Docu_Test')

Folder exists
Folder exists


C:\Users\ebryaga\AppData\Local\Temp\ipykernel_15696\2898220153.py:47: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


/html/body/form/b[2]/table[1]/tbody/tr[3]/td[1]/a
Cannot Download ['/html/body/form/b[2]/table[1]/tbody/tr[3]/td[1]/a', '/html/body/form/b[2]/table[1]/tbody/tr[4]/td[1]/a', '/html/body/form/b[2]/table[1]/tbody/tr[5]/td[1]/a', '/html/body/form/b[2]/table[2]/tbody/tr[3]/td[1]/a', '/html/body/form/b[2]/table[3]/tbody/tr[3]/td[1]/a']
Whoops
Whoops


C:\Users\ebryaga\AppData\Local\Temp\ipykernel_15696\2898220153.py:47: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


/html/body/form/b[2]/table[1]/tbody/tr[3]/td[1]/a
/html/body/form/b[2]/table[2]/tbody/tr[3]/td[1]/a
/html/body/form/b[2]/table[3]/tbody/tr[3]/td[1]/a
C:\Users\ebryaga\Downloads\batchacc (5).tiff
/html/body/form/b[2]/table[3]/tbody/tr[4]/td[1]/a
C:\Users\ebryaga\Downloads\batchacc (6).tiff
/html/body/form/b[2]/table[4]/tbody/tr[3]/td[1]/a
C:\Users\ebryaga\Downloads\batchacc (7).tiff
/html/body/form/b[2]/table[4]/tbody/tr[4]/td[1]/a
/html/body/form/b[2]/table[5]/tbody/tr[3]/td[1]/a
Cannot Download ['/html/body/form/b[2]/table[1]/tbody/tr[3]/td[1]/a', '/html/body/form/b[2]/table[2]/tbody/tr[3]/td[1]/a', '/html/body/form/b[2]/table[3]/tbody/tr[3]/td[1]/a', '/html/body/form/b[2]/table[3]/tbody/tr[4]/td[1]/a', '/html/body/form/b[2]/table[4]/tbody/tr[3]/td[1]/a', '/html/body/form/b[2]/table[4]/tbody/tr[4]/td[1]/a', '/html/body/form/b[2]/table[5]/tbody/tr[3]/td[1]/a', '/html/body/form/b[2]/table[5]/tbody/tr[4]/td[1]/a']


C:\Users\ebryaga\AppData\Local\Temp\ipykernel_15696\2898220153.py:47: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


/html/body/form/b[2]/table[1]/tbody/tr[3]/td[1]/a
/html/body/form/b[2]/table[1]/tbody/tr[4]/td[1]/a
/html/body/form/b[2]/table[2]/tbody/tr[3]/td[1]/a
C:\Users\ebryaga\Downloads\batchacc (9).tiff
/html/body/form/b[2]/table[2]/tbody/tr[4]/td[1]/a
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\Users\ebryaga\Downloads\1c345684-dddf-4a29-82f9-94918a651673.tmp
C:\User

In [28]:
np.nan

nan